In [1]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import json 
import os

### Load data

In [2]:
# Load annotations
annotations = json.load(open(os.path.join('Project1.2', 'annotations.json')))

In [3]:
# Load images
data_path = '/dtu/datasets1/02514/data_wastedetection/'
images = {}
for image_id in range(len(annotations['images'])):
    images[image_id] = cv2.imread(os.path.join(data_path, annotations['images'][image_id]['file_name']))

In [4]:
# Get bounding box for every image 
annotations['bounding_boxes'] = {}
for image_id in range(len(annotations['images'])):
    bbxstart = 10000
    bbxfin = 0
    bbystart = 10000
    bbyfin = 0
    for id, value in enumerate(annotations['annotations'][image_id]['segmentation'][0]):
        if id % 2 == 0:
            if bbxstart > value: bbxstart = value
            if bbxfin < value: bbxfin = value
        else:
            if bbystart > value: bbystart = value
            if bbyfin < value: bbyfin = value

    annotations['bounding_boxes'][image_id] = [int(bbxstart), int(bbystart), int(bbxfin), int(bbyfin)]

### Resize images and bounding boxes

In [5]:
def resize(image, BB_start, BB_width, BB_height, size=(224, 224)):
    ### resize image
    img_resized = cv2.resize(image.copy(), size, interpolation = cv2.INTER_AREA)
    
    ### resize BB
    # get x and y ratio
    lx = size[0]/image.shape[1]
    ly = size[1]/image.shape[0]
    
    # get new (x,y), width, height
    BB_new_start = (int(BB_start[0]*lx), int(BB_start[1]*ly))
    BB_new_width = int(BB_width*ly)
    BB_new_height = int(BB_height*lx)
    
    return img_resized, BB_new_start, BB_new_width, BB_new_height

In [7]:
# Resize images and bounding boxes
resized_images = []
annotations['new_bounding_boxes'] = {}
for image_id in images:
    
    # Resize images and bounding boxes
    width = annotations['bounding_boxes'][image_id][2] - annotations['bounding_boxes'][image_id][0]
    height = annotations['bounding_boxes'][image_id][3] - annotations['bounding_boxes'][image_id][1]
    new_image, new_bb, new_width, new_height = resize(images[image_id], (annotations['bounding_boxes'][image_id][0], annotations['bounding_boxes'][image_id][1]), width, height)
    
    # write resized images
    resized_images.append(new_image)

    # write new bounding boxes
    annotations['new_bounding_boxes'][image_id] = [int(new_bb[0]), int(new_bb[1]), int(new_bb[0] + new_width), int(new_bb[1] + new_height)]